<a href="https://colab.research.google.com/github/adithyxnpramod/TrainingSessionsForStudents/blob/main/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten

In [22]:
# 2. Mount Google Drive
drive.mount('/content/drive')

# Define the folder path in Google Drive
drive_folder = "/content/drive/MyDrive/mnist_data"  # Folder to store MNIST dataset
os.makedirs(drive_folder, exist_ok=True)  # Create the folder if it doesn't exist

# Paths for saving CSV files
train_csv_path = os.path.join(drive_folder, "mnist_train.csv")
test_csv_path = os.path.join(drive_folder, "mnist_test.csv")

# Check if the dataset already exists in Google Drive
if os.path.exists(train_csv_path) and os.path.exists(test_csv_path):
    print("Dataset already exists in Google Drive. Loading from CSV files...")
else:
    # 3. Download the MNIST dataset
    print("Downloading MNIST dataset...")
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # Normalize the pixel values to [0, 1]
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Flatten the images (28x28 -> 784)
    x_train_flat = x_train.reshape(x_train.shape[0], -1)
    x_test_flat = x_test.reshape(x_test.shape[0], -1)

    # One-hot encode the labels
    y_train_onehot = tf.keras.utils.to_categorical(y_train, num_classes=10)
    y_test_onehot = tf.keras.utils.to_categorical(y_test, num_classes=10)

    # Combine features and labels into a single DataFrame
    train_df = pd.DataFrame(np.hstack([x_train_flat, y_train_onehot]))
    test_df = pd.DataFrame(np.hstack([x_test_flat, y_test_onehot]))

    # Save to CSV in Google Drive
    train_df.to_csv(train_csv_path, index=False, header=False)
    test_df.to_csv(test_csv_path, index=False, header=False)

    print(f"Training data saved to {train_csv_path}")
    print(f"Testing data saved to {test_csv_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset already exists in Google Drive. Loading from CSV files...


In [23]:
# 3. Read the CSV files back into memory
import pandas as pd # Importing pandas here
print("Loading dataset from Google Drive...")
train_data = pd.read_csv(train_csv_path, header=None)
test_data = pd.read_csv(test_csv_path, header=None)

# Separate features and labels
x_train_csv = train_data.iloc[:, :-10].values  # First 784 columns are features
y_train_csv = train_data.iloc[:, -10:].values  # Last 10 columns are one-hot labels

x_test_csv = test_data.iloc[:, :-10].values
y_test_csv = test_data.iloc[:, -10:].values

Loading dataset from Google Drive...


In [24]:
# 4. Define the model
model = keras.Sequential([
    Flatten(input_shape=(28 * 28,)),  # Flatten input (784 features)
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# 5. Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 6. Train the model
model.fit(x_train_csv, y_train_csv, epochs=5, batch_size=32)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8772 - loss: 0.4375
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9640 - loss: 0.1219
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9770 - loss: 0.0796
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9818 - loss: 0.0574
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9868 - loss: 0.0437


In [25]:
# 7. Evaluate on test data
loss, accuracy = model.evaluate(x_test_csv, y_test_csv, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

# 8. Make predictions (optional)
predictions = model.predict(x_test_csv)
predicted_labels = tf.argmax(predictions, axis=1)

# Display first 10 predictions
print(predicted_labels[:10])

Test loss: 0.08199544996023178
Test accuracy: 0.9743000268936157
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
tf.Tensor([7 2 1 0 4 1 4 9 5 9], shape=(10,), dtype=int64)
